работа с текстом
необходимо, чтобы текстовая колонка находилась под именем 'text'

In [1]:
import pandas as pd
import numpy as np

import re

в x_test уже лемматизированный текст, но для проверки работы, использую его

In [21]:
test = pd.read_csv('Data/x_test.csv', index_col=0)

In [22]:
test.rename(columns = {'lemmas_string':'text'}, inplace = True )
test.head()

,text
217,продажа успешный кейс компактный динамичный ка...
28197,макгрегор vs ислам хороший бой хороший выступл...
8279,выиграть медведев борьба лидерство рейтинг гот...
19020,неподсказать хороший рацион соревнование борье...
8920,алексей ягудин считать гениальный старт карьер...


In [3]:
def clean_text(text):
    text = text.lower()
    # убираем цифры
    clean_text = re.sub("[0-9]", "", text)
    # убираем ссылки
    clean_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", text)
    # оставляем только буквенные символы
    clean_text = re.sub("[^а-яa-z]"," ", clean_text)
    # убираем лишние пробелы
    clean_text = re.sub(" +", " ", clean_text)
    return clean_text

выбор списка стоп-слов

In [5]:
import nltk
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_nltk = list(stopwords.words('russian'))

print('Количество стоп-слов библиотеки nltk =',len(stop_words_nltk))

Количество стоп-слов библиотеки nltk = 151


[nltk_data] Downloading package stopwords to /home/marina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


нашла другой список стоп-слов из https://countwordsfree.com/stopwords/russian

In [23]:
# opening the file in read mode
my_file = open("Data/stop_words_russian.txt", "r")

# reading the file
stopwords = my_file.read()

# replacing end splitting the text
# when newline ('\n') is seen.
stopwords_list = stopwords.split("\n")
print('Количество стоп-слов из countwordsfree.com =',len(stopwords_list))
my_file.close()

Количество стоп-слов из countwordsfree.com = 421


In [8]:
stop_words_of_sport = ['трансляция','видео','канал']
stopwords_list+=stop_words_of_sport

In [9]:
!pip install pymorphy2

In [10]:
from pymorphy2 import MorphAnalyzer

lemmatyzer = MorphAnalyzer()

In [11]:
def post_to_wordlist(post_sentence, remove_stopwords=False ):
    # разбиваем предложение на слова по символу пробела
    words = post_sentence.split()
    if remove_stopwords: # убираем стоп-слова
        words = [w for w in words if not w in remove_stopwords and len(w)>1]

    lemma_words = [lemmatyzer.parse(w)[0].normal_form for w in words]

    if remove_stopwords: # убираем стоп-слова
        lemma_words_nostops = [w for w in lemma_words if not w in remove_stopwords]

    return(lemma_words_nostops)

In [19]:
def prepose_text(df_text):
    df_text['clean_text'] = df_text['text'].map(clean_text)
    df_no_adv = df_text.drop_duplicates(subset='clean_text')
    df_no_adv['tokenized_text'] = df_no_adv['clean_text'].apply(lambda row: post_to_wordlist(row, stopwords_list))
    df_no_adv['lemmas_string'] = df_no_adv['tokenized_text'].apply(lambda row: ' '.join(row))
    return df_no_adv

In [20]:
test_lemma = prepose_text(test)
test_lemma.head()

/tmp/ipykernel_55223/906220149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_adv['tokenized_text'] = df_no_adv['clean_text'].apply(lambda row: post_to_wordlist(row, stopwords_list))
/tmp/ipykernel_55223/906220149.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_adv['lemmas_string'] = df_no_adv['tokenized_text'].apply(lambda row: ' '.join(row))


,text,clean_text,tokenized_text,lemmas_string
217,продажа успешный кейс компактный динамичный ка...,продажа успешный кейс компактный динамичный ка...,"[продажа, успешный, кейс, компактный, динамичн...",продажа успешный кейс компактный динамичный ка...
28197,макгрегор vs ислам хороший бой хороший выступл...,макгрегор vs ислам хороший бой хороший выступл...,"[макгрегор, vs, ислам, хороший, бой, хороший, ...",макгрегор vs ислам хороший бой хороший выступл...
8279,выиграть медведев борьба лидерство рейтинг гот...,выиграть медведев борьба лидерство рейтинг гот...,"[выиграть, медведев, борьба, лидерство, рейтин...",выиграть медведев борьба лидерство рейтинг гот...
19020,неподсказать хороший рацион соревнование борье...,неподсказать хороший рацион соревнование борье...,"[неподсказать, хороший, рацион, соревнование, ...",неподсказать хороший рацион соревнование борье...
8920,алексей ягудин считать гениальный старт карьер...,алексей ягудин считать гениальный старт карьер...,"[алексей, ягудин, считать, гениальный, старт, ...",алексей ягудин считать гениальный старт карьер...
